In [1]:
using Rocket
using ReactiveMP
using GraphPPL
using BenchmarkTools
using Distributions
using MacroTools

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1278


In [2]:
@model function smoothing(n, x0, c::ConstVariable, P::ConstVariable)
    
    x_prior ~ NormalMeanVariance(mean(x0), cov(x0)) 

    x = randomvar(n)
    y = datavar(Float64, n)
    
    x_prev = x_prior
    
    for i in 1:n
        x[i] ~ x_prev + c
        y[i] ~ NormalMeanVariance(x[i], P)
        
        x_prev = x[i]
    end

    return x, y
end

smoothing (generic function with 1 method)

In [3]:
P = 1.0

n = 500
data = collect(1:n) + rand(Normal(0.0, sqrt(P)), n);

In [4]:
function inference(data, x0, P)
    n = length(data)
    
    _, (x, y) = smoothing(n, x0, 1.0, P);

    x_buffer  = buffer(Marginal, n)
    marginals = getmarginals(x)
    
    subscription = subscribe!(marginals, x_buffer)
    
    update!(y, data)
    
    unsubscribe!(subscription)
    
    return getvalues(x_buffer)
end

inference (generic function with 1 method)

In [5]:
x0_prior = NormalMeanVariance(0.0, 10000.0)
@benchmark res = inference($data, $x0_prior, $P)

BenchmarkTools.Trial: 
  memory estimate:  17.60 MiB
  allocs estimate:  296267
  --------------
  minimum time:     24.254 ms (0.00% GC)
  median time:      26.341 ms (0.00% GC)
  mean time:        29.872 ms (13.82% GC)
  maximum time:     48.857 ms (18.74% GC)
  --------------
  samples:          168
  evals/sample:     1